In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO

import json
from tqdm import tqdm_notebook as tqdm

In [2]:
url_search = 'https://rating.chgk.info/api/tournaments.json/search?name={}'
url_tournament = 'https://rating.chgk.info/api/tournaments/{}/list'
url_recaps = 'https://rating.chgk.info/api/tournaments/{idtournament}/recaps/{idteam}'
url_player = 'https://rating.chgk.info/api/players/{}'

Change the search query here

In [3]:
search = 'чемпионат мира'

In [25]:
response = requests.get(url_search.format(search)).text
response = json.load(StringIO(response))
tournaments = pd.DataFrame(response['items'])
tournaments = tournaments.loc[(tournaments.name.str.lower() == search) | 
                              tournaments.name.str.endswith('Этап 4. Группа A') |
                              tournaments.name.str.endswith('Финал. Группа А') | 
                              tournaments.name.str.endswith('Финал'),
                              :]

In [26]:
tournaments

,archive,date_archived_at,date_end,date_start,idtournament,name,type_name
0,None,None,2002-07-08 00:00:00,2002-07-07 00:00:00,712,Чемпионат мира,Обычный
1,None,None,2003-08-18 00:00:00,2003-08-13 00:00:00,8,Чемпионат мира,Обычный
2,None,None,2004-08-01 00:00:00,2004-07-31 00:00:00,54,Чемпионат мира,Обычный
3,None,None,2005-09-10 00:00:00,2005-09-10 00:00:00,116,Чемпионат мира,Обычный
4,None,None,2006-08-27 00:00:00,2006-08-27 00:00:00,188,Чемпионат мира,Обычный
5,None,None,2007-08-25 00:00:00,2007-08-24 00:00:00,272,Чемпионат мира,Обычный
6,None,None,2008-09-21 00:00:00,2008-09-17 00:00:00,385,Чемпионат мира,Обычный
7,None,None,2010-11-14 00:00:00,2010-11-14 00:00:00,1533,Чемпионат мира. Финал,Обычный
8,None,None,2011-09-18 00:00:00,2011-09-17 00:00:00,1893,Чемпионат мира,Обычный
11,None,None,2012-09-09 00:00:00,2012-09-09 00:00:00,2169,Чемпионат мира. Финал,Обычный


In [35]:
len(tournaments)

17

In [27]:
def get_player_stat_tournament(idtournament):
    tournament_results = json.load(StringIO(requests.get(url_tournament.format(idtournament)).text))
    player_results = []
    
    for team_result in tournament_results:
        position = float(team_result['position'])
        team_recaps = json.load(StringIO(requests.get(url_recaps.format(idtournament=idtournament, idteam=team_result['idteam'])).text))
        # Collect player stats
        for player in team_recaps:
            player_result = {'idplayer': player['idplayer'],
                             'idteam': team_result['idteam'],
                             'team_name': team_result['base_name'],
                             'position': position,
                             'questions': team_result['questions_total']}
            player_results.append(player_result)

    return pd.DataFrame(player_results)

In [28]:
tournaments.date_end[0].split('-')[0]

'2002'

In [29]:
player_results = []
for i, r in tqdm(tournaments.iterrows(), total=len(tournaments)):
    res = get_player_stat_tournament(r.idtournament)
    res['year'] = r.date_end.split('-')[0]
    player_results.append(res)

In [30]:
all_players = pd.concat(player_results)
all_players.head()

,idplayer,idteam,position,questions,team_name,year
0,707,2,4.0,0,Афина,2002
1,13551,2,4.0,0,Афина,2002
2,15442,2,4.0,0,Афина,2002
3,25882,2,4.0,0,Афина,2002
4,30475,2,4.0,0,Афина,2002


Ever played this tournament

In [31]:
len(all_players.idplayer.unique())

895

Played max times

In [32]:
def get_name(idplayer):
    response = json.load(StringIO(requests.get(url_player.format(idplayer)).text))[0]
    return ' '.join([response['surname'], response['name'], response['patronymic']])

def get_names(players):
    return [get_name(i) for i in tqdm(players)]

In [105]:
tournament_counts.index.value_counts().sum()

895

In [33]:
tournament_counts = all_players.groupby('idplayer').count().iloc[:,0].sort_values(ascending=False)
max_times_players = tournament_counts[tournament_counts == tournament_counts.max()]
max_times_players

idplayer
18036    17
18332    17
Name: idteam, dtype: int64

In [34]:
get_names(max_times_players.index.tolist())

['Левандовский Михаил Ильич', 'Либер Александр Витальевич']

In [106]:
# n = 30
n = 895
n_times_players_names = get_names(tournament_counts.index.tolist()[:n])
n_times_players = pd.DataFrame(tournament_counts)
n_times_players.loc[:n, 'name'] = n_times_players_names

In [37]:
n_times_players.head(30)

,idteam,name
idplayer,,
18036,17,Левандовский Михаил Ильич
18332,17,Либер Александр Витальевич
6212,16,Выменец Юрий Яковлевич
27403,16,Руссо Максим Михайлович
22935,15,Новиков Илья Сергеевич
20691,14,Мереминский Станислав Григорьевич
31190,14,Тальянский Илья Николаевич
9808,14,Друзь Александр Абрамович
25882,13,Поташев Максим Оскарович


In [97]:
n_times_players

,idteam,name
idplayer,,
18036,17,Левандовский Михаил Ильич
18332,17,Либер Александр Витальевич
6212,16,Выменец Юрий Яковлевич
27403,16,Руссо Максим Михайлович
22935,15,Новиков Илья Сергеевич
20691,14,Мереминский Станислав Григорьевич
31190,14,Тальянский Илья Николаевич
9808,14,Друзь Александр Абрамович
25882,13,Поташев Максим Оскарович


Count medals

In [38]:
medalists = all_players.loc[all_players.position < 4,['idplayer', 'position']].reset_index(drop=True)
medalists.position = np.array(medalists.position, dtype=np.uint8)
len(medalists)

322

In [39]:
medalists.head()

,idplayer,position
0,2620,1
1,8909,1
2,9808,1
3,9809,1
4,9810,1


In [58]:
medalists_aggregated = pd.get_dummies(medalists, columns=['position']).groupby('idplayer').sum().sort_values(['position_{}'.format(i) for i in range(1,4)], ascending=False)
medalists_aggregated['name'] = get_names(medalists_aggregated.index.tolist())
medalists_aggregated

,position_1,position_2,position_3,name
idplayer,,,,
6212,4,4,4,Выменец Юрий Яковлевич
18036,4,4,3,Левандовский Михаил Ильич
18332,4,4,3,Либер Александр Витальевич
27403,3,4,1,Руссо Максим Михайлович
28751,3,2,1,Семушин Иван Николаевич
4270,3,2,1,Брутер Александра Владимировна
30152,3,1,1,Сорожкин Артём Сергеевич
9808,3,1,0,Друзь Александр Абрамович
3207,2,4,3,Берёзкина Ольга Игоревна


In [59]:
medalists_aggregated_copy = medalists_aggregated.copy()

In [94]:
medalists_aggregated = medalists_aggregated_copy.copy()

In [48]:
def make_clickable(id_name):
    id_name = id_name.split()
    idplayer = id_name[0]
    player = ' '.join(id_name[1:])
    player = '<a href="https://rating.chgk.info/player/{}">{}</a>'.format(idplayer, player)
    return player

In [95]:
medalists_aggregated['position_s'] = medalists_aggregated.loc[:,'position_1':'position_3'].sum(axis=1)
medalists_aggregated = medalists_aggregated.iloc[:, [3,0,1,2,4]].reset_index().sort_values(['position_{}'.format(i) for i in range(1,4)], ascending=False)
medalists_aggregated['idplayer_name'] = medalists_aggregated.idplayer + ' ' + medalists_aggregated.name
medalists_aggregated = medalists_aggregated.drop(['idplayer', 'name'], axis=1)
medalists_aggregated = medalists_aggregated.iloc[:, [-1,0,1,2,3]]
medalists_aggregated = medalists_aggregated.set_index(np.arange(1, len(medalists_aggregated) + 1))
medalists_aggregated = medalists_aggregated.style.format(make_clickable, subset='idplayer_name')
medalists_aggregated

,idplayer_name,position_1,position_2,position_3,position_s
1,Выменец Юрий Яковлевич,4,4,4,12
2,Левандовский Михаил Ильич,4,4,3,11
3,Либер Александр Витальевич,4,4,3,11
4,Руссо Максим Михайлович,3,4,1,8
5,Семушин Иван Николаевич,3,2,1,6
6,Брутер Александра Владимировна,3,2,1,6
7,Сорожкин Артём Сергеевич,3,1,1,5
8,Друзь Александр Абрамович,3,1,0,4
9,Берёзкина Ольга Игоревна,2,4,3,9
10,Матвеев Михаил Леонидович,2,4,2,8


In [96]:
with open("champions.html", "w", encoding='utf-8') as f:
    f.write(medalists_aggregated.render())

In [26]:
medalists_aggregated.to_csv('./medalists_kest.csv')